In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
spacelike_delimiters = [' ','\xa0']

def myLikesCountParser(stringg):
    for delimiter in spacelike_delimiters:
        stringg = stringg.replace(delimiter,' ').replace(',','.')
    num = float(stringg.partition(' ')[0])
    if "тыс" in stringg:
        num = num*10**3
    elif "млн" in stringg:
        num = num*10**6
    return int(num)

In [3]:
df=pd.read_json('C:/Users/VG/Saved Games/Desktop/dippractice/data/politics/ru/putinspeech.json', lines=True, encoding="utf-8")
df.head(3)

,author,channel,cid,heart,photo,text,time,time_parsed,votes
0,Реакция на Видео,UCeTNUKV1tFt6k0b3keF_qmg,Ugy_UKOlpOHFEdXX4L54AaABAg,False,https://yt3.ggpht.com/O1jdSk1dKPz_8Fk0Ho302vQQ...,"Кто-то захотел повоевать, а нам просто охота м...",6 дней назад,1.645588e+09,"3,2 тыс."
1,soiung toiue,UCKLPCycdk9PzFWvpUBvGxRg,UgzuBfFtV8ETQYQdYBZ4AaABAg,False,https://yt3.ggpht.com/ytc/AKedOLT1iLNAz93fk4I5...,"🇷🇺🤝🇺🇦 мирного небо, нам братья и сестры. Пусть...",2 дня назад,1.645934e+09,8
2,Vlad Romanovsi,UCAHTYeqQJUJEbXymt_-W32Q,UgwpwO0RC53aDPKQ7414AaABAg,False,https://yt3.ggpht.com/ytc/AKedOLQrw-HsReDKHQXg...,Братья украинцы мы против войны . То что проис...,5 дней назад,1.645675e+09,"1,2 тыс."


In [4]:
df = df.drop(['photo', 'channel', 'time'], axis=1)
df["time_parsed"] = df["time_parsed"].apply(lambda x: datetime.fromtimestamp(x))
df["votes"] = df["votes"].apply(lambda x: myLikesCountParser(x))

df['isBaseComment'] = False
df["RepliesCount"] = 0
df["RepliesID"] = [[] for i in range(0,df.count()[0])]

In [5]:
commentsdict = dict()
n = df.count()[0]

In [6]:
%%time
for index, row in df.iterrows():
    temp = row["cid"].partition('.')
    
    if temp[0] not in commentsdict:
        commentsdict[temp[0]] = []
        
    if temp[2] == '': 
        df.loc[index,"isBaseComment"] = True
    else:
        commentsdict[temp[0]].append(index)

Wall time: 2.3 s


In [7]:
%%time
df.loc[df["isBaseComment"] == True, "RepliesID"] = df.loc[df["isBaseComment"] == True, "cid"].apply(lambda x: commentsdict[x])
df["RepliesCount"] = df["RepliesID"].apply(lambda x: len(x))

Wall time: 9.97 ms


In [8]:
df = df.rename(columns={"author": "Nickname", "heart" : "LikedByAuthor", "text": "CommentText", "time_parsed": "Date", "votes" : "LikesCount"})
df = df.drop(["cid"], axis=1)

In [9]:
df.head(10)

,Nickname,LikedByAuthor,CommentText,Date,LikesCount,isBaseComment,RepliesCount,RepliesID
0,Реакция на Видео,False,"Кто-то захотел повоевать, а нам просто охота м...",2022-02-23 06:51:52.449724,3200,True,175,"[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 3..."
1,soiung toiue,False,"🇷🇺🤝🇺🇦 мирного небо, нам братья и сестры. Пусть...",2022-02-27 06:51:52.451693,8,True,0,[]
2,Vlad Romanovsi,False,Братья украинцы мы против войны . То что проис...,2022-02-24 06:51:52.452690,1200,True,179,"[195, 196, 197, 198, 199, 200, 201, 202, 203, ..."
3,Богдана Богданівна,False,Берегите друг-друга....я против войньі между н...,2022-02-24 06:51:52.453687,275,True,18,"[374, 375, 376, 377, 378, 379, 380, 381, 382, ..."
4,Алексей Х.,False,Знаете одну простую истину? За все всегда расп...,2022-02-23 06:51:52.454684,491,True,11,"[392, 393, 394, 395, 396, 397, 398, 399, 400, ..."
5,bocoy noiu,False,"Один народ сказал:""Моя земля!"" Другой сказал-""...",2022-02-28 10:51:52.456679,3,True,0,[]
6,Jonny D,False,Лучший бой - тот которого удалось избежать.\nВ...,2022-02-23 06:51:52.457676,1400,True,34,"[403, 404, 405, 406, 407, 408, 409, 410, 411, ..."
7,Наталия Мирина,False,"Господи,спаси и сохрани,всех людей!!! Нет войн...",2022-02-23 06:51:52.458674,602,True,15,"[437, 438, 439, 440, 441, 442, 443, 444, 445, ..."
8,Сания Сания азазза,False,"🇷🇺🤝🇺🇦 мирного небо, нам братья и сестры. Пусть...",2022-02-23 06:51:52.459672,389,True,14,"[452, 453, 454, 455, 456, 457, 458, 459, 460, ..."
9,Кирилл Ложников,False,"Нет войне!!! Мы за мир!!!!\nГосподи, спаси и с...",2022-02-25 06:51:52.461668,82,True,5,"[466, 467, 468, 469, 470]"


In [10]:
df.to_csv(path_or_buf='C:/Users/VG/Saved Games/Desktop/dippractice/data/politics/ru/putinspeech.csv')